## Вграждания

В предишния пример работихме с високодименсионални вектори на чанта от думи с дължина `vocab_size`, като изрично преобразувахме от нискодименсионални вектори на позиционно представяне в разредено едноразрядно представяне. Това едноразрядно представяне не е ефективно по отношение на паметта, освен това всяка дума се третира независимо от останалите, т.е. едноразрядно кодираните вектори не изразяват никаква семантична прилика между думите.

В този модул ще продължим да изследваме набора от данни **News AG**. За начало, нека заредим данните и вземем някои дефиниции от предишния тефтер.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Какво е вграждане?

Идеята на **вграждането** е да представим думите чрез плътни вектори с по-ниска размерност, които по някакъв начин отразяват семантичното значение на думата. По-късно ще обсъдим как да създаваме смислени вграждания на думи, но засега нека просто мислим за вграждането като начин за намаляване на размерността на векторите на думите.

Така че, слой за вграждане ще приема дума като вход и ще произвежда изходен вектор със зададен `embedding_size`. В известен смисъл, той е много подобен на слоя `Linear`, но вместо да приема вектор с едно горещо кодиране, ще може да приема номер на дума като вход.

Използвайки слой за вграждане като първи слой в нашата мрежа, можем да преминем от модел на чанта с думи към модел на **чанта с вграждания**, където първо преобразуваме всяка дума в текста в съответното вграждане, а след това изчисляваме някаква агрегатна функция върху всички тези вграждания, като например `sum`, `average` или `max`.

![Изображение, показващо класификатор с вграждане за пет последователни думи.](../../../../../translated_images/bg/embedding-classifier-example.b77f021a7ee67eee.webp)

Нашата невронна мрежа за класификация ще започне със слой за вграждане, след това слой за агрегиране и линеен класификатор върху него:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Работа с променлив размер на последователността

В резултат на тази архитектура, минипартидите за нашата мрежа трябва да бъдат създадени по определен начин. В предишния модул, когато използвахме bag-of-words, всички BoW тензори в една минипартида имаха еднакъв размер `vocab_size`, независимо от действителната дължина на текстовата последователност. След като преминем към word embeddings, ще се окажем с променлив брой думи във всяка текстова извадка, и при комбинирането на тези извадки в минипартиди ще трябва да приложим известно запълване.

Това може да се направи, като използваме същата техника за предоставяне на функция `collate_fn` към източника на данни:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Обучение на класификатор с вградени представяния

Сега, след като сме дефинирали подходящия dataloader, можем да обучим модела, използвайки функцията за обучение, която дефинирахме в предишния модул:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Забележка**: Тук тренираме само за 25k записа (по-малко от една пълна епоха) заради времето, но можете да продължите обучението, да напишете функция за обучение за няколко епохи и да експериментирате с параметъра на скоростта на обучение, за да постигнете по-висока точност. Трябва да можете да достигнете точност от около 90%.


### Слой EmbeddingBag и представяне на променливи по дължина последователности

В предишната архитектура трябваше да запълним всички последователности до еднаква дължина, за да ги включим в минипартида. Това не е най-ефективният начин за представяне на последователности с променлива дължина - друг подход би бил използването на **вектор на отместванията**, който съдържа отместванията на всички последователности, съхранени в един голям вектор.

![Изображение, показващо представяне на последователност с отмествания](../../../../../translated_images/bg/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: На изображението по-горе е показана последователност от символи, но в нашия пример работим с последователности от думи. Въпреки това, основният принцип на представяне на последователности с вектор на отмествания остава същият.

За да работим с представяне чрез отмествания, използваме слоя [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Той е подобен на `Embedding`, но приема вектор със съдържание и вектор с отмествания като вход, и също така включва слой за осредняване, който може да бъде `mean`, `sum` или `max`.

Ето модифицирана мрежа, която използва `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

За да подготвим набора от данни за обучение, трябва да предоставим функция за преобразуване, която ще подготви вектора на отместване:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Имайте предвид, че за разлика от всички предишни примери, нашата мрежа сега приема два параметъра: вектор на данни и вектор на отместване, които са с различни размери. По същия начин, нашият модул за зареждане на данни също ни предоставя 3 стойности вместо 2: както текстовите, така и векторите на отместване се предоставят като характеристики. Следователно, трябва леко да коригираме нашата функция за обучение, за да се погрижим за това:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Семантични вграждания: Word2Vec

В предишния пример слоят за вграждане на модела се научи да преобразува думите във векторно представяне, но това представяне нямаше особено семантично значение. Би било добре да се научи такова векторно представяне, при което подобни думи или синоними да съответстват на вектори, които са близо един до друг според някаква векторна дистанция (например евклидово разстояние).

За да постигнем това, трябва предварително да обучим модела за вграждане върху голяма колекция от текст по специфичен начин. Един от първите методи за обучение на семантични вграждания се нарича [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Той се базира на две основни архитектури, които се използват за създаване на разпределено представяне на думите:

 - **Непрекъсната торба с думи** (CBoW) — при тази архитектура обучаваме модела да предсказва дума въз основа на заобикалящия контекст. Даден е n-грам $(W_{-2},W_{-1},W_0,W_1,W_2)$, целта на модела е да предскаже $W_0$ от $(W_{-2},W_{-1},W_1,W_2)$.
 - **Непрекъснат скип-грам** е противоположен на CBoW. Моделът използва заобикалящия прозорец от контекстни думи, за да предскаже текущата дума.

CBoW е по-бърз, докато скип-грам е по-бавен, но се справя по-добре с представянето на редки думи.

![Изображение, показващо алгоритмите CBoW и Skip-Gram за преобразуване на думи във вектори.](../../../../../translated_images/bg/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

За да експериментираме с вграждания word2vec, предварително обучени върху набора от данни Google News, можем да използваме библиотеката **gensim**. По-долу намираме думите, които са най-близки до 'neural'.

> **Note:** Когато за първи път създавате векторни представяния на думи, изтеглянето им може да отнеме известно време!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Можем също така да изчислим векторни вграждания от думата, които да се използват при обучението на класификационен модел (показваме само първите 20 компонента на вектора за яснота):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Страхотното при семантичните вграждания е, че можете да манипулирате векторното кодиране, за да промените семантиката. Например, можем да поискаме да намерим дума, чиято векторна репрезентация да бъде възможно най-близка до думите *крал* и *жена*, и възможно най-далеч от думата *мъж*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

И CBoW, и Skip-Grams са „предсказателни“ вграждания, тъй като вземат предвид само локалните контексти. Word2Vec не използва глобалния контекст.

**FastText** надгражда Word2Vec, като научава векторни представяния за всяка дума и за символните n-грамове, които се намират в думата. Стойностите на представянията се осредняват в един вектор на всяка стъпка от обучението. Въпреки че това добавя значително допълнително изчисление към предварителното обучение, то позволява на вгражданията на думи да кодират информация за поддуми.

Друг метод, **GloVe**, използва идеята за матрица на съвместна поява и прилага невронни методи за разлагане на матрицата на съвместна поява в по-изразителни и нелинейни векторни представяния на думи.

Можете да експериментирате с примера, като промените вгражданията на FastText и GloVe, тъй като gensim поддържа няколко различни модела за вграждане на думи.


## Използване на предварително обучени вграждания в PyTorch

Можем да променим горния пример, за да предварително запълним матрицата в слоя за вграждане със семантични вграждания, като Word2Vec. Трябва да имаме предвид, че речниците на предварително обучените вграждания и нашия текстов корпус вероятно няма да съвпадат, затова ще инициализираме теглата за липсващите думи със случайни стойности:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Сега нека обучим нашия модел. Имайте предвид, че времето за обучение на модела е значително по-дълго в сравнение с предишния пример, поради по-големия размер на слоя за вграждане и съответно много по-големия брой параметри. Също така, заради това може да се наложи да обучим модела си върху повече примери, ако искаме да избегнем прекомерното напасване.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

В нашия случай не наблюдаваме значително увеличение на точността, което вероятно се дължи на доста различни речници.  
За да преодолеем проблема с различните речници, можем да използваме едно от следните решения:  
* Преобучаване на word2vec модела върху нашия речник  
* Зареждане на нашия набор от данни с речника от предварително обучен word2vec модел. Речникът, използван за зареждане на набора от данни, може да бъде зададен по време на зареждането.  

Вторият подход изглежда по-лесен, особено защото PyTorch `torchtext` рамката съдържа вградена поддръжка за вграждания. Можем, например, да създадем речник, базиран на GloVe, по следния начин:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Заредената речникова структура има следните основни операции:  
* Речникът `vocab.stoi` ни позволява да преобразуваме дума в нейния индекс в речника  
* `vocab.itos` прави обратното - преобразува число в дума  
* `vocab.vectors` е масивът от векторите на вграждане, така че за да получим вграждането на дума `s`, трябва да използваме `vocab.vectors[vocab.stoi[s]]`  

Ето пример за манипулиране на вграждания, за да демонстрираме уравнението **kind-man+woman = queen** (трябваше да коригирам коефициента малко, за да проработи):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

За да обучим класификатора, използвайки тези вграждания, първо трябва да кодираме нашия набор от данни, използвайки речника на GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Както видяхме по-горе, всички векторни вграждания се съхраняват в матрицата `vocab.vectors`. Това прави изключително лесно зареждането на тези тегла в теглата на слоя за вграждане чрез просто копиране:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Сега нека обучим нашия модел и да видим дали ще получим по-добри резултати:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Една от причините, поради които не наблюдаваме значително увеличение на точността, е фактът, че някои думи от нашия набор от данни липсват в предварително обучената GloVe лексика и следователно те са на практика игнорирани. За да преодолеем този факт, можем да обучим собствени вграждания върху нашия набор от данни.


## Контекстуални вграждания

Едно от основните ограничения на традиционните предварително обучени вграждания като Word2Vec е проблемът с разграничаването на значенията на думите. Докато предварително обучените вграждания могат да уловят част от значението на думите в контекст, всяко възможно значение на дадена дума се кодира в едно и също вграждане. Това може да създаде проблеми в последващи модели, тъй като много думи, като например думата "play", имат различни значения в зависимост от контекста, в който се използват.

Например, думата "play" в тези две различни изречения има съвсем различно значение:
- Отидох на **пиеса** в театъра.
- Джон иска да **играе** с приятелите си.

Предварително обучените вграждания по-горе представят и двете значения на думата "play" в едно и също вграждане. За да преодолеем това ограничение, трябва да изградим вграждания, базирани на **езиков модел**, който е обучен върху голям корпус от текст и *знае* как думите могат да се комбинират в различни контексти. Обсъждането на контекстуални вграждания е извън обхвата на този урок, но ще се върнем към тях, когато говорим за езикови модели в следващия модул.



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
